<a href="https://colab.research.google.com/github/dtobi59/nsga3-mammography-hpo-main/blob/master/colab_mammography_hpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NSGA-III Hyperparameter Optimization for Mammography Classification

Multi-objective hyperparameter optimization for deep learning models in breast cancer detection.

**Objectives:**
- Maximize: Sensitivity, Specificity, AUC
- Minimize: Model Size

## 1. Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

## 2. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. Clone Repository & Install Dependencies

In [ ]:
# Clone the repository
!git clone https://github.com/dtobi59/nsga3-mammography-hpo-main.git
%cd nsga3-mammography-hpo-main

In [ ]:
# Install required packages
!pip install -q -r requirements.txt

## 4. Download Kaggle VinDr-Mammo PNG Dataset (RECOMMENDED)

**This dataset is 5-10x faster than DICOM!**

Follow these 4 steps to download the dataset:

In [ ]:
# STEP 1: Install Kaggle CLI
!pip install -q kaggle

print("Kaggle CLI installed!")
print("\nNext: Upload your kaggle.json file in the cell below.")

In [ ]:
# STEP 2: Upload your kaggle.json file
# Get it from: https://www.kaggle.com/settings -> Create New API Token

from google.colab import files

print("Please upload your kaggle.json file:")
print("(Go to https://www.kaggle.com/settings and click 'Create New API Token')")
print()

uploaded = files.upload()

# Setup Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("\nKaggle credentials configured!")

In [ ]:
# STEP 3: Download the dataset
print("Downloading Kaggle VinDr-Mammo PNG dataset...")
print("Dataset size: ~8.25 GB - this will take a few minutes\n")

!kaggle datasets download -d shantanughosh/vindr-mammogram-dataset-dicom-to-png

print("\nDownload complete!")

In [ ]:
# STEP 4: Extract to Google Drive
print("Extracting dataset to Google Drive...")
print("This will be saved to: /content/drive/MyDrive/kaggle_vindr_data\n")

!unzip -q vindr-mammogram-dataset-dicom-to-png.zip -d /content/drive/MyDrive/kaggle_vindr_data

print("\n" + "="*70)
print("DATASET READY!")
print("="*70)
print("\nLocation: /content/drive/MyDrive/kaggle_vindr_data")
print("\nThe dataset is now ready to use!")
print("Continue to the next section to configure and load it.")
print("="*70)

## 5. Configure Dataset Path

**After downloading the dataset above**, the paths below should work with the default settings.

If you're using a different dataset (VinDr DICOM or INbreast), uncomment the appropriate option.

In [ ]:
# Configure your dataset

# OPTION A: Kaggle VinDr-Mammo PNG (Recommended - 5-10x faster!)
# Use this if you downloaded the dataset using cells above
DATASET_NAME = "kaggle_vindr_png"
DATA_ROOT = "/content/drive/MyDrive/kaggle_vindr_data"

# OPTION B: Original VinDr-Mammo DICOM (slower)
# Use this if you already have VinDr-Mammo DICOM dataset
# DATASET_NAME = "vindr"
# DATA_ROOT = "/content/drive/MyDrive/vindr-mammo"

# OPTION C: INbreast DICOM (slower)
# DATASET_NAME = "inbreast"
# DATA_ROOT = "/content/drive/MyDrive/inbreast"

# Output directory (where results will be saved)
OUTPUT_DIR = "/content/drive/MyDrive/nsga3_outputs"

print("="*70)
print("DATASET CONFIGURATION")
print("="*70)
print(f"Dataset: {DATASET_NAME}")
print(f"Data root: {DATA_ROOT}")
print(f"Output directory: {OUTPUT_DIR}")
print("="*70)

## 6. Test Your Setup (Optional but Recommended)

Run this quick test to verify everything is working before starting the full workflow.

In [ ]:
# Run setup verification test
!python test_colab_setup.py --data_root {DATA_ROOT}

## 7. Load Dataset

In [ ]:
from dataset import prepare_dataset

print(f"Loading {DATASET_NAME} dataset from {DATA_ROOT}...")

train_paths, train_labels, val_paths, val_labels = prepare_dataset(
    dataset_name=DATASET_NAME,
    data_root=DATA_ROOT
)

print(f"\nDataset loaded successfully!")
print(f"Train samples: {len(train_paths)}")
print(f"Validation samples: {len(val_paths)}")
print(f"Train labels distribution: {sum(train_labels)} malignant, {len(train_labels) - sum(train_labels)} benign")
print(f"Val labels distribution: {sum(val_labels)} malignant, {len(val_labels) - sum(val_labels)} benign")

## 8. Visualize Sample Images

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from dataset import load_dicom_image
from PIL import Image

def visualize_samples(image_paths, labels, dataset_name, n_samples=6):
    """
    Visualize sample mammography images.
    """
    # Select random samples
    n_samples = min(n_samples, len(image_paths))
    indices = np.random.choice(len(image_paths), n_samples, replace=False)

    # Create subplot grid
    n_cols = 3
    n_rows = (n_samples + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))

    if n_rows == 1:
        axes = axes.reshape(1, -1)

    fig.suptitle(f'{dataset_name} Dataset Samples', fontsize=16, fontweight='bold')

    for idx, ax_idx in enumerate(range(n_samples)):
        row = ax_idx // n_cols
        col = ax_idx % n_cols
        ax = axes[row, col]

        sample_idx = indices[idx]
        img_path = image_paths[sample_idx]
        label = labels[sample_idx]

        try:
            # Load image (supports both DICOM and PNG)
            if img_path.lower().endswith(('.dcm', '.dicom')):
                image = load_dicom_image(img_path, apply_clahe=True)
            else:
                # PNG/JPG image
                img = Image.open(img_path)
                if img.mode != 'RGB':
                    img = img.convert('RGB')
                image = np.array(img)

            # Display
            ax.imshow(image, cmap='gray')
            label_text = 'MALIGNANT' if label == 1 else 'BENIGN'
            label_color = 'red' if label == 1 else 'green'
            ax.set_title(f'{label_text}', fontsize=12, fontweight='bold', color=label_color)
            ax.axis('off')

            # Add filename as subtitle
            filename = os.path.basename(img_path)
            ax.text(0.5, -0.05, filename[:30] + '...' if len(filename) > 30 else filename,
                   ha='center', va='top', transform=ax.transAxes,
                   fontsize=8, style='italic')

        except Exception as e:
            ax.text(0.5, 0.5, f'Error loading image:\n{str(e)[:50]}',
                   ha='center', va='center', transform=ax.transAxes)
            ax.axis('off')

    # Hide extra subplots
    for idx in range(n_samples, n_rows * n_cols):
        row = idx // n_cols
        col = idx % n_cols
        axes[row, col].axis('off')

    plt.tight_layout()
    plt.show()

# Visualize training samples
print("=" * 70)
print("TRAINING SET SAMPLES")
print("=" * 70)
visualize_samples(train_paths, train_labels, 'Training', n_samples=6)

print("\n" + "=" * 70)
print("VALIDATION SET SAMPLES")
print("=" * 70)
visualize_samples(val_paths, val_labels, 'Validation', n_samples=6)

## 9. Configure Optimization

Adjust these parameters based on your computational budget:
- **pop_size**: Population size (more = better exploration, longer runtime)
- **n_generations**: Number of generations (more = better convergence)
- **epochs**: Training epochs per evaluation (reduce for faster testing)

In [ ]:
from config import ExperimentConfig
from dataclasses import asdict

# Load default config
config = ExperimentConfig()

# Customize NSGA-III parameters
config.nsga3.pop_size = 20        # Recommended: 20-50 for quick runs, 100+ for thorough search
config.nsga3.n_generations = 10   # Recommended: 5-10 for testing, 20-50 for production

# Training epochs per evaluation (reduce for faster testing)
TRAINING_EPOCHS = 5  # Recommended: 5 for testing, 10-20 for production

print(f"Optimization Configuration:")
print(f"  Population size: {config.nsga3.pop_size}")
print(f"  Generations: {config.nsga3.n_generations}")
print(f"  Epochs per evaluation: {TRAINING_EPOCHS}")
print(f"  Total evaluations: ~{config.nsga3.pop_size * config.nsga3.n_generations}")
print(f"\nSearching hyperparameters:")

# Convert dataclass to dictionary for iteration
hp_space_dict = asdict(config.hyperparameter_space)
for param, values in hp_space_dict.items():
    print(f"  {param}: {values}")

## 10. Create Evaluation Function

In [ ]:
from training import full_evaluation

def make_eval_fn(tp, tl, vp, vl, epochs=5):
    """
    Creates an evaluation function for the optimizer.
    """
    def eval_fn(hp_config):
        hp_config = hp_config.copy()
        hp_config['epochs'] = epochs
        return full_evaluation(
            hp_config,
            tp, tl, vp, vl,
            device='cuda',  # Use GPU
            verbose=True
        )
    return eval_fn

eval_function = make_eval_fn(
    train_paths, train_labels,
    val_paths, val_labels,
    epochs=TRAINING_EPOCHS
)

print("Evaluation function created successfully!")

## 11. Run Optimization

This will take some time depending on your configuration.

**Estimated runtime:**
- Small (pop=20, gen=5, epochs=5): ~30-60 minutes
- Medium (pop=50, gen=10, epochs=10): ~2-4 hours
- Large (pop=100, gen=20, epochs=20): ~8-12 hours

In [ ]:
from optimization import run_optimization
import time

# Set random seed for reproducibility
RANDOM_SEED = 42

print(f"Starting optimization with seed={RANDOM_SEED}...\n")
start_time = time.time()

results = run_optimization(
    hp_space=config.hyperparameter_space,
    nsga_config=config.nsga3,
    eval_function=eval_function,
    output_dir=OUTPUT_DIR,
    seed=RANDOM_SEED
)

elapsed_time = time.time() - start_time
print(f"\n{'='*70}")
print(f"Optimization completed in {elapsed_time/3600:.2f} hours")
print(f"{'='*70}")

## 12. View Results

In [ ]:
# Display Pareto front solutions
print(f"\nFound {len(results['pareto_configs'])} Pareto-optimal solutions\n")
print("="*100)

for i, (cfg, obj) in enumerate(zip(results['pareto_configs'], results['pareto_F'])):
    print(f"\nSolution {i+1}:")
    print("-" * 100)

    # Architecture details
    print(f"  Architecture:")
    print(f"    Backbone: {cfg['backbone']}")
    print(f"    Unfreeze: {cfg['unfreeze_strategy']}")
    print(f"    Dropout: {cfg['dropout_rate']:.2f}")
    print(f"    FC Hidden: {cfg['fc_hidden_size']}")

    # Training details
    print(f"  Training:")
    print(f"    Optimizer: {cfg['optimizer']}")
    print(f"    Learning Rate: {cfg['learning_rate']:.6f}")
    print(f"    Batch Size: {cfg['batch_size']}")
    print(f"    Loss: {cfg['loss_function']}")

    # Performance metrics
    print(f"  Performance:")
    print(f"    Sensitivity: {obj[0]:.4f}")
    print(f"    Specificity: {obj[1]:.4f}")
    print(f"    AUC: {obj[2]:.4f}")
    print(f"    Model Size: {obj[3]:.2f}M parameters")

## 13. Visualize Pareto Front

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract objectives
pareto_F = np.array(results['pareto_F'])

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Pareto Front - Trade-off Analysis', fontsize=16, fontweight='bold')

# Sensitivity vs Specificity
axes[0, 0].scatter(pareto_F[:, 0], pareto_F[:, 1], c='blue', s=100, alpha=0.6)
axes[0, 0].set_xlabel('Sensitivity', fontsize=12)
axes[0, 0].set_ylabel('Specificity', fontsize=12)
axes[0, 0].set_title('Sensitivity vs Specificity')
axes[0, 0].grid(True, alpha=0.3)

# AUC vs Model Size
axes[0, 1].scatter(pareto_F[:, 2], pareto_F[:, 3], c='green', s=100, alpha=0.6)
axes[0, 1].set_xlabel('AUC', fontsize=12)
axes[0, 1].set_ylabel('Model Size (M params)', fontsize=12)
axes[0, 1].set_title('AUC vs Model Size')
axes[0, 1].grid(True, alpha=0.3)

# Sensitivity vs Model Size
axes[1, 0].scatter(pareto_F[:, 0], pareto_F[:, 3], c='purple', s=100, alpha=0.6)
axes[1, 0].set_xlabel('Sensitivity', fontsize=12)
axes[1, 0].set_ylabel('Model Size (M params)', fontsize=12)
axes[1, 0].set_title('Sensitivity vs Model Size')
axes[1, 0].grid(True, alpha=0.3)

# Overall metrics distribution
metrics = ['Sensitivity', 'Specificity', 'AUC', 'Size (M)']
avg_values = pareto_F.mean(axis=0)
axes[1, 1].bar(metrics, avg_values, color=['blue', 'green', 'red', 'purple'], alpha=0.6)
axes[1, 1].set_ylabel('Average Value', fontsize=12)
axes[1, 1].set_title('Average Objective Values')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/pareto_front_visualization.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\nVisualization saved to {OUTPUT_DIR}/pareto_front_visualization.png")

## 14. Save Best Configuration

In [ ]:
import json

# Find solution with best AUC
best_auc_idx = np.argmax(pareto_F[:, 2])
best_auc_config = results['pareto_configs'][best_auc_idx]
best_auc_objectives = pareto_F[best_auc_idx]

print(f"Best AUC Configuration:")
print(f"  AUC: {best_auc_objectives[2]:.4f}")
print(f"  Sensitivity: {best_auc_objectives[0]:.4f}")
print(f"  Specificity: {best_auc_objectives[1]:.4f}")
print(f"\nConfiguration: {json.dumps(best_auc_config, indent=2)}")

# Save to file
with open(f"{OUTPUT_DIR}/best_auc_config.json", 'w') as f:
    json.dump({
        'config': best_auc_config,
        'objectives': {
            'sensitivity': float(best_auc_objectives[0]),
            'specificity': float(best_auc_objectives[1]),
            'auc': float(best_auc_objectives[2]),
            'model_size_M': float(best_auc_objectives[3])
        }
    }, f, indent=2)

print(f"\nBest configuration saved to {OUTPUT_DIR}/best_auc_config.json")

## 15. Results Summary

All results have been saved to your Google Drive:
- Pareto-optimal configurations
- Optimization history
- Visualizations
- Best configuration JSON

You can now:
1. Download the results from Google Drive
2. Use the best configuration to train a final model
3. Deploy the model for inference
4. Run additional experiments with different parameters